All the news have been extracted from https://business.financialpost.com/

In [20]:
import requests
import urllib
import sys
import numpy  as np
import pandas as pd

from bs4 import BeautifulSoup

# Extract web links according to a signature and the number of pages


In [21]:
def get_internal_links(ur):
    
    info  = BeautifulSoup(requests.get(ur,allow_redirects = True).content,'html.parser').find_all("div",{"class":"article-card__details"})
    links  = [ "https://financialpost.com" + i.a['href'] for i in info]

    return(links)   

In [22]:
def extract_link_of_news(k_word,n_of_page):
    link= "https://financialpost.com/search/?search_text="+ k_word + "&date_range=-3650d&sort=score&from=" + str((n_of_page-1)*10)
    news_df                  = pd.DataFrame()    
    
    news_df["internal_urls"]  = get_internal_links(link)
    news_df["principal_url"]  = link    
    news_df["n_of_page"]      = n_of_page
    return(news_df[["n_of_page","principal_url","internal_urls"]])

In [23]:
n_of_pages = 170
df = pd.concat([extract_link_of_news("apple",i) for i in range(1,n_of_pages+1)],ignore_index = True)
df

,n_of_page,principal_url,internal_urls
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/app...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/mov...
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/app...
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/fir...
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/qua...
...,...,...,...
1695,170,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/cyb...
1696,170,https://financialpost.com/search/?search_text=...,https://financialpost.com/investing/apple-back...
1697,170,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/app...
1698,170,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/personal-...


In [32]:
n_of_pages = 135
df_cisco = pd.concat([extract_link_of_news("cisco",i) for i in range(1,n_of_pages+1)],ignore_index = True)
df_cisco

,n_of_page,principal_url,internal_urls
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
...,...,...,...
1345,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/executive/leadership...
1346,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/dissectin...
1347,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/executive/executive-...
1348,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/news/economy/public-...


# Extract the date and text from each link 

In [33]:
def extract_date_and_text(ur):
    url = ur
    
    try:
        soup         = BeautifulSoup(requests.get(url).content, 'html.parser')
        g = 0
    
    except:   
        the_type, the_value, the_traceback = sys.exc_info()
        g = 1
    

    if(g==0):
        flag= True
        if(soup.find("section",class_ = "article-content__content-group")):
            text_of_news = soup.find("section",class_ = "article-content__content-group").find_all("p")
            flag=False
        else:
            text_of_news = "NO TEXT"

        
        if(flag):
            final_date = ""
            final_text =""
        else:
            final_date= soup.find("span",class_="published-date__since").text
            get_text_vec = [i.text for i in text_of_news]
            final_text = "".join(get_text_vec)
        #final_date   = soup.find("span",class_="published-date__since").text["datetime"]
        
        
    else: final_date = "";final_text = ""
    
    return(final_date,final_text)

In [34]:
a = np.vectorize(extract_date_and_text)
test = df_cisco[0:10].copy()
test["internals_dates"],test["internals_text"] = a(test["internal_urls"][0:10])
test

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 10, 2021",TORONTO — Softchoice (TSX: SFTC) today announc...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 02, 2020","TORONTO — Softchoice , a leading North America..."
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 04, 2021","FRANKLIN, Tenn. — Elevēo announces the integr..."
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 20, 2019","Saving time, energy and money, Ingram Micro’s ..."
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Apr 14, 2021",News Summary:SAN FRANCISCO — Cisco AppDynamics...
5,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Feb 25, 2021",Business Unit will leverage Cisco technologies...
6,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jun 18, 2020","For the first time in 30 years, Cisco Live was..."
7,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Apr 01, 2021","Cisco Canada’s new president, Shannon Leininge..."
8,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"May 12, 2021","DENVER — Berkana Resources Corporation, a lead..."
9,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Jun 16, 2020",NS1 + Cisco deliver comprehensive traffic man...


In [35]:
a = np.vectorize(extract_date_and_text)
df_cisco["internals_dates"],df_cisco["internals_text"] = a(df_cisco["internal_urls"])

In [36]:
df_cisco

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 10, 2021",TORONTO — Softchoice (TSX: SFTC) today announc...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 02, 2020","TORONTO — Softchoice , a leading North America..."
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 04, 2021","FRANKLIN, Tenn. — Elevēo announces the integr..."
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 20, 2019","Saving time, energy and money, Ingram Micro’s ..."
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Apr 14, 2021",News Summary:SAN FRANCISCO — Cisco AppDynamics...
...,...,...,...,...,...
1345,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/executive/leadership...,"Mar 17, 2014",OTTAWA — In the five years since Nortel went b...
1346,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/dissectin...,"Jun 06, 2017",SAN FRANCISCO — When a withered Yahoo is absor...
1347,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/executive/executive-...,"Nov 12, 2020",Good Morning!Seven months into the pandemic an...
1348,135,https://financialpost.com/search/?search_text=...,https://financialpost.com/news/economy/public-...,"May 01, 2012","Police officers and firefighters in San Jose, ..."


In [37]:
df_cisco.loc[df_cisco["internals_text"] == "NO TEXT"]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [39]:
len(df_cisco.loc[df_cisco["internals_text"] == ""])

7

In [41]:
df_cisco.loc[df_cisco["internals_text"] == ""]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
335,34,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Oct 15, 2020",
392,40,https://financialpost.com/search/?search_text=...,https://financialpost.com/business-trends/why-...,"Aug 26, 2020",
546,55,https://financialpost.com/search/?search_text=...,https://financialpost.comhttps://nationalpost....,,
632,64,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Aug 06, 2019",
635,64,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jul 30, 2019",
672,68,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Aug 12, 2020",
726,73,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"May 27, 2020",


In [1]:
df_cisco.to_pickle("data sets/custom_cisco.pkl")

NameError: name 'df_cisco' is not defined

In [ ]:
u=df['internal_urls'][131]
u

In [16]:
extract_date_and_text(u)

('Apr 06, 2021', '')

In [17]:
s = BeautifulSoup(requests.get(u).content, 'html.parser')

In [18]:
text=s.find_all("section",class_ = "article-content__content-group")

In [19]:
res=[i.find_all("p") for i in text]

In [20]:
res

[[],
 [<p>RESEARCH TRIANGLE PARK, N.C. — Today, Lenovo (HKSE: 992) (ADR: LNVGY) Infrastructure Solutions Group (ISG) announces the next-generation Lenovo ThinkSystem servers showcasing a unique balance of performance, security and efficiency – all built on 3rd Gen Intel Xeon Scalable processors and PCIe Gen4. As companies of all sizes continue to work on solving real-world challenges – they require powerful infrastructure solutions to help them gain faster insights and remain competitive. With this new generation of ThinkSystem solutions, Lenovo introduces innovations for real-world workloads including high performance computing (HPC), artificial intelligence (AI), modeling and simulation, cloud, virtual desktop infrastructure (VDI) and advanced analytics. </p>,
  <p> “Our next-generation ThinkSystem Server platform delivers a unique balance of performance, security, and efficiency,” said Kamran Amini, Vice President and General Manager of Infrastructure Solutions Platforms, Lenovo Inf

In [21]:
df['internals_text'][0]

'The parking guidance system solution is developed in collaboration with IntelBENGALURU, India — L&T Technology Services Limited (BSE: 540115, NSE: LTTS), a leading global engineering services company, announced today that it worked with Intel Corporation to develop an outdoor smart parking solution. The solution is powered by the Intel distribution of the OpenVINO Toolkit to run AI inferencing models on Intel Xeon scalable processors and Intel Movidius VPUs. With edge AI capabilities, the solution aims to redefine the outdoor smart parking experience in public areas across the globe. '